In [1]:
import time
import pandas as pd
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException        

def init_system(path_to_chromedriver):
    reload(sys)
    sys.setdefaultencoding('utf-8')
    driver = init_driver(path_to_chromedriver)
    return driver

def init_driver(path):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    #chrome_options.add_argument('--proxy-server=107.174.48.114:8989')
    chrome_options.add_argument("--start-maximized");
    browser = webdriver.Chrome(executable_path = path,chrome_options=chrome_options)
    return browser

def check_exists_by_xpath(driver,xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

def fetch_nextpage(driver):
    if check_exists_by_xpath(driver,"//*[@id='result-details']/main/div/nav/span/a/i[@class = 'icon-caret-right']"):
        elements= WebDriverWait(driver, 200).until(
            EC.presence_of_all_elements_located((By.XPATH, "//*[contains(@class,'next')]/a[contains(@href,'page')]"))
        )
        link = elements[0].get_attribute("href")
        driver.get(link)
        i = 0
        while "search" not in driver.page_source:
            driver.get(link)
            time.sleep(0.8)
            i += 1
            if(i==50):
                driver.close()
                return 0
    else:
        return 0
    return 1
            
def select_propertyforsell(driver):
    element = WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#content > main > div.home-criteria > div > div.tabset-tabs-container > div.tabset-tab.selected"))
    )
    element.click()
    
def select_advancedoptions(driver):
    element = WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#form > div:nth-child(4) > div.criteria-field.options > div > div:nth-child(2)")))
    element.click()
    
def select_activeproperty(driver):
    element = WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#status > option:nth-child(1)")))
    element.click()

def search(driver,key):
    element = WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.ID, "zip"))
    )
    element.send_keys(key)

    element = WebDriverWait(driver, 200).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#form > div.below_fold_toggle_sale.hidden > div.criteria-field.search-button > button"))
    )
    element.click()
    
def scrape_page(driver,column_names):
    L = []
    item_list = WebDriverWait(driver, 200).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='details row']"))
       )
    
    for item in item_list:
        name = str(item.find_element_by_class_name('details-title').find_elements_by_css_selector('a')[0].text)
        price = str(item.find_element_by_class_name('price').text)
        l =item.find_elements_by_xpath("ul//li//ul//li")
        bed = None
        bath = None
        size = None
        for s in l:
            if("bed" in s.text): 
                bed = str(s.text)
            if("ft" in s.text):
                size = str(s.text)
            if("bath" in s.text):
                bath = str(s.text)
        L.append(tuple((name,price,bed,bath,size)))
        df = pd.DataFrame(L, columns=column_names)
    return df

def extract_number(string):
    if string is None or string == '':
        return 0
    return int(filter(str.isdigit, string))


In [2]:
def fecth_data(path_to_chromedriver,url,zip_code,page,column_names):
    
    driver = init_system(path_to_chromedriver)
    driver.get(url)
    select_propertyforsell(driver)
    select_advancedoptions(driver)
    select_activeproperty(driver)
    search(driver,zip_code)

    time.sleep(2)
    df = pd.DataFrame(columns = column_names)
    if("No results matched your search" in driver.page_source ):
        driver.close()
        return df
    
    for i in range(1,page):
        df = df.append(scrape_page(driver,column_names))
        if(not fetch_nextpage(driver)):
            break

    df = df.reset_index()
    df.to_csv(zip_code+".csv")
    driver.close()
    return df

In [3]:
def main():
    path_to_chromedriver = '../chromedriver'
    url = 'http://streeteasy.com/'
    column_names = ['Property','Price','Bedrooms','Bathrooms','Size']
    for i in range(10030,10035):
        page_data = fecth_data(path_to_chromedriver,url,str(i),50,column_names) 
        #page_data.head()
        time.sleep(2)

if __name__ == "__main__":
    main()

In [4]:
import os.path
column_names = ['Property','Price','Bedrooms','Bathrooms','Size','Zip']
data = pd.DataFrame()
for i in range(10001,10020):
    filename = str(i)+".csv"
    if os.path.isfile(filename):
        d = pd.read_csv(filename)
        d['Zip'] = str(i)
        data = data.append(d)
data = data[column_names]
data = data[data['Property'].notnull()]
#data['Price'] = [(extract_number(x) if x is not None else 0) for x in data['Price']]
data.to_csv("Property Sample Data.csv")
